### Day 16

### Part 1:
- Turn hex into binary
- Process binary with some pretty complex rules that I won't type out
    - Each binary "packet" can be made of subpackets
    - Each packet/subpacket needs to be processed according to some rules defined by their header
    - Can have arbitrary number of extra bits at the end
- Sum the version number of each packet/subpacket

In [1]:
class Decoder(object):
    def __init__(self, fname=None, verbose=False):
        
        if fname is not None:
            with open(fname, "r") as f:
                data = f.read().splitlines()
            
            self.hex_packet = data
        self.total_packet_version = 0
        self.verbose = verbose
        
    def hex_to_bin(self, hex_packet):
        """Convert the packet to binary"""
        bin_output = ""
        for digit in hex_packet:
            bin_output += bin(int(digit,16))[2:].zfill(4)
        return bin_output
        
    def process_packet_type_4(self, packet):
        """Return the literal value in a packet or subpacket"""
        # Literal value
        literal = ""

        # Loop through the number and build the value
        start_ix = 0
        still_looking = True
        while still_looking:
            # Take next 5 digits
            digits = packet[start_ix:start_ix+5]

            # If it starts with a zero this is the end
            if digits[0] == "0":
                still_looking = False

            literal += digits[1:]
            start_ix += 5

        # Convert to decimal
        output = int(literal,2)
        
        remaining_packet = packet[start_ix:]
        
        return output, remaining_packet
    
        
    def decode_packet(self, hex_packet, is_binary=False):
        """Decode and process the packet."""
        # Convert from hex to binary
        if is_binary:
            packet = hex_packet
        else:
            packet = self.hex_to_bin(hex_packet)
        
        if self.verbose:
            print("Current packet:",packet)

        packet_version = int(packet[0:3],2)
        packet_type = int(packet[3:6],2)
        
        self.total_packet_version += packet_version
        remaining_packet = ""

        if packet_type == 4:
            output, remaining_packet = self.process_packet_type_4(packet[6:])
            if self.verbose:
                print("Literal packet:",output)
            
        else:
            # Operator packet
            packet_length_type_id = packet[6]
            if packet_length_type_id == "0":
                tot_len = int(packet[7:22],2)
                if self.verbose:
                    print("Operator packet, subpacket length",tot_len,"bits")
                
                # Process the subpackets
                remaining_subpacket = packet[22:22+tot_len]
                remaining_packet = packet[22+tot_len:]# Not part of this group of subpackets
                output = []
                while len(remaining_subpacket) > 0:
                    result = self.decode_packet(remaining_subpacket, is_binary=True)
                    subpacket_output, remaining_subpacket = result
                    output.append(subpacket_output)
                
            else:
                num_subpackets = int(packet[7:18],2)
                if self.verbose:
                    print("Operator packet with",num_subpackets,"subpackets")
                
                # Process the subpackets
                remaining_packet = packet[18:]
                output = []
                for ix in range(num_subpackets):
                    result = self.decode_packet(remaining_packet, is_binary=True)
                    subpacket_output, remaining_packet = result
                    output.append(subpacket_output)
                    
            # If it's an operator packet we need to process the subpackets
            # (For part 2)
            output = self.process_operator_packet_result(output, packet_type)
                
        return int(output),remaining_packet

    def process_operator_packet_result(self,packet_vals, packet_type):
        """Handle the output of an operator packet"""
        if packet_type == 0:
            # Sum the results
            return sum(packet_vals)
        
        elif packet_type == 1:
            # Multiply the results
            output = 1
            for val in packet_vals:
                output *= val
        
        elif packet_type == 2:
            # Take minimum value
            output = min(packet_vals)
            
        elif packet_type == 3:
            # Take maximum value
            output = max(packet_vals)
            
        elif packet_type == 5:
            # 1 if val[0] > val[1]
            output = 1*(packet_vals[0] > packet_vals[1])
        
        elif packet_type == 6:
            # 1 if val[0] < val[1]
            output = 1*(packet_vals[0] < packet_vals[1])

        elif packet_type == 7:
            # 1 if val[0] == val[1]
            output = 1*(packet_vals[0] == packet_vals[1])
        return output

In [2]:
# Test inputs
hex_packet = "D2FE28" # Literal 2021
hex_packet = "38006F45291200" # Operator type id 0 with subpackets 10 and 20
hex_packet = "EE00D40C823060" # Operator type id 1 with subpackets 1,2,3
hex_packet = "8A004A801A8002F478" # version sum 16
hex_packet = "620080001611562C8802118E34" # version sum 12
hex_packet = "C0015000016115A2E0802F182340" # version sum 23
hex_packet = "A0016C880162017C3686B18A3D4780" # version sum 31
d = Decoder(verbose=False)
print(d.decode_packet(hex_packet))
print(d.total_packet_version)

(54, '0000000')
31


In [3]:
# Puzzle input
d = Decoder(fname="inputs/day16_input.dat",verbose=False)
d.decode_packet(d.hex_packet[0]);
print(d.total_packet_version)

843


### Part 2:
- Previous part ignored the processing of the packets/subpackets
- Rules for adding, multiplying, >, <, min, max etc.
- Add the operator processing to the previous code

In [4]:
# Test inputs
hex_packet = "C200B40A82" # Result 3
hex_packet = "04005AC33890" # Result 54
hex_packet = "880086C3E88112" # Result 7
hex_packet = "CE00C43D881120" # Result 9
hex_packet = "D8005AC2A8F0" # Result 1
hex_packet = "F600BC2D8F" # Result 0
hex_packet = "9C005AC2F8F0" # Result 0
hex_packet = "9C0141080250320F1802104A08" # Result 1
d = Decoder(verbose=False)
print(d.decode_packet(hex_packet))

(1, '00')


In [5]:
# Puzzle input
d = Decoder(fname="inputs/day16_input.dat",verbose=False)
d.decode_packet(d.hex_packet[0])

(5390807940351, '00')